In [1]:
import torch, json
from math import floor
import pandas as pd
import numpy as np

from FileManager.dataManager import dataManager
from AnalyzeTools.models import autoregressive_integrated_moving_average, linear_regression, support_vector_regression, random_forest, gradient_boosting
from AnalyzeTools.prepare import data_split, model_eval, pathForSavingModels
from AnalyzeTools.preprocess import preprocessData
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 123


In [2]:
period = 'Day'
future_step = 1
params_path = f'./Models'

product_object = json.load(open("./File information.json", "r", encoding='utf8'))

all_experiments= []
for product in product_object.keys():
    for raw_file_name in  product_object[product].keys():
        for product_type in product_object[product][raw_file_name]['product_types']:
            for target in product_object[product][raw_file_name]['targets']:
                all_experiments.append([product, raw_file_name, product_type, target])

In [3]:
n = 0
experiment = all_experiments[n]
product, raw_file_name, product_type, target = experiment
print(f"Product: {product}\nRaw file name: {raw_file_name}\nProduct_type: {product_type}\ntarget: {target}")

Product: pork
Raw file name: (중)경략가격집계 - 소,돼지
Product_type: 돼지 온도체
target: MAX_COST_AMT


In [4]:
df, product_and_product_type, product_attribute = dataManager(raw_file_name, product, product_type, target)

if len(df) == 0:
    raise ValueError("No data!")

df, input_features = preprocessData(df, 'date', target)
test_size = 0.1

train_x_axis = df['date'][:-1*floor(len(df) * test_size)].values if type(test_size) == float else df['date'][:-1*test_size]
predictions_x_axis = df['date'][-1*floor(len(df) * test_size):].values if type(test_size) == float else df['date'][-1*test_size:]


-->Feature scores:
                 Features        Scores
6  DEFECT_MAX_COST_AMT  6.811574e+10
1         MIN_COST_AMT  1.222592e+04
5  DEFECT_MIN_COST_AMT  1.222443e+04
2         SUM_COST_AMT  4.538033e+02
7  DEFECT_SUM_COST_AMT  4.537454e+02
8    DEFECT_SUM_WEIGHT  2.279391e+01
3           SUM_WEIGHT  2.277102e+01
4           DEFECT_CNT  1.084715e+01
0                  CNT  1.083094e+01

-->TOP K features:
   ['MIN_COST_AMT', 'SUM_COST_AMT', 'DEFECT_MIN_COST_AMT', 'DEFECT_MAX_COST_AMT']

-->Final features:
  ['MIN_COST_AMT', 'SUM_COST_AMT']


In [5]:
# prepare dataset for statistics and Macnhine models
ml_split_params = {'Model': 'ML', 'Future': future_step}
X_train, X_test, y_train, y_test, input_scaler, output_scaler = data_split(df, input_features, target, test_size, scaling=True, **ml_split_params)

''' Input data into models and Evaluate model results '''
ml_searchCV_params = {
    'base_dir': params_path,
    'product_and_product_type': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'predict_type': 'single',
    'period': 'Day',
    'step': future_step,
    'save': True
}
stdout = True
vis = True

(2548, 8)
(2547, 9)
X_train: (2293, 3) y_train: (2293,) X_test: (254, 3) y_test: (254,)


# Statistics

In [6]:
# print("\nARIMA")
# arima_predictions = autoregressive_integrated_moving_average(y_train, y_test)
# model_eval(y_test, arima_predictions, stdout=stdout, vis=vis)

In [7]:
print("\nLinear Regression")
lr, _ = linear_regression(X_train, y_train)
lr_predictions = lr.predict(X_test)
model_eval(y_test, lr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Linear Regression
MAPE: 0.056253028695720735 R square: 0.6611838736052357


# Machine learning

In [8]:
print("\nSupport Vector Regression")
svr, _ = support_vector_regression(X_train, y_train, search=True, **ml_searchCV_params)
svr_predictions = svr.predict(X_test)
model_eval(y_test, svr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

[I 2022-12-01 19:55:01,013] A new study created in memory with name: no-name-fa5f1ddd-8555-4031-8f5b-a43048566e77



Support Vector Regression
--> Start searching best parameters!

Best parameter for SVR is:
  {'C': 100, 'gamma': 0.01, 'epsilon': 0.0001, 'kernel': 'rbf'}
MAPE: 0.056331146223482495 R square: 0.6368600567885624


In [9]:
print("\nRandom Forest")
rf, _ = random_forest(X_train, y_train, search=True, **ml_searchCV_params)
rf_predictions = rf.predict(X_test)
model_eval(y_test, rf_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Random Forest
--> Start searching best parameters!

Best parameter for Random forest is:
  {'n_estimators': 210, 'max_depth': 7, 'max_features': 3, 'min_samples_leaf': 9, 'min_samples_split': 7}
MAPE: 0.06965813426054283 R square: 0.5520618563102921


In [10]:
print("\nGradient Boosting")
gb, _ = gradient_boosting(X_train, y_train, search=True, **ml_searchCV_params)
gb_predictions = gb.predict(X_test)
model_eval(y_test, gb_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})


Gradient Boosting
--> Start searching best parameters!

Best parameter for Gradient Boosting is:
  {'n_estimators': 90, 'max_depth': 1, 'max_features': 3, 'min_samples_leaf': 9, 'min_samples_split': 5}
MAPE: 0.07420156375662883 R square: 0.4788460772288633


# Deep learning

In [11]:
data = df.copy()

# if future_step != 1:
#     dl_target = f'{target}_lead_{future_step}'
#     data[dl_target] = data[target].shift(-1 * (future_step - 1))
#     data = data[:-1 * (future_step - 1)]

data['time_idx'] = range(len(data))
data['group'] = product

training_cutoff = floor(len(data) * (1-test_size)) if type(test_size) == float else len(data) - test_size

max_prediction_length = 1 # In the case of point forecasting, this parameter is 1 by default
max_encoder_length = 30 # 7, 14, 30, 60, 120
batch_size = 64

group = ['group']
time_varying_known_categoricals = ['month', 'week']
time_varying_unknown_categoricals = []
time_varying_known_reals = ['time_idx']
time_varying_unknown_reals = input_features + [target]

# if future_step != 1:
#     time_varying_unknown_reals = input_features + [target, dl_target]
#     target_arg = dl_target
# else:
#     time_varying_unknown_reals = input_features + [target]
#     target_arg = target

In [12]:
dl_searchCV_params = {
    'base_dir': params_path,
    'product_and_product_type': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'predict_type': 'single',
    'period': 'Day',
    'step': future_step,
    'save': True
}

In [13]:
print("\nLSTM")
training_params = {'max_epochs': 100, 'n_trials': 30}
saving_dir = pathForSavingModels('LSTM', **dl_searchCV_params)
lstm, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    saving_dir,
    'LSTM',
    training_params,
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
lstm_predictions = lstm.predict(val_dataloader)
model_eval(actuals, lstm_predictions, predictions_x_axis, stdout=True, vis=True)


LSTM


c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES

Epoch 17: 100%|██████████| 39/39 [00:02<00:00, 16.22it/s, loss=0.0269, v_num=0, train_loss_step=0.030, train_loss_epoch=0.0268, val_loss=0.042]  
MAPE: 0.04202990606427193 R square: 0.8244478309960569


In [14]:
print("\nGRU")
training_params = {'max_epochs': 100, 'n_trials': 30}
saving_dir = pathForSavingModels('GRU', **dl_searchCV_params)
gru, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    saving_dir,
    'GRU',
    training_params
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
gru_predictions = gru.predict(val_dataloader)

model_eval(actuals, gru_predictions, predictions_x_axis, stdout=True, vis=True)

c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



GRU


c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES

Epoch 23: 100%|██████████| 39/39 [00:02<00:00, 14.04it/s, loss=0.0329, v_num=0, train_loss_step=0.0422, train_loss_epoch=0.0325, val_loss=0.0444]
MAPE: 0.044377293437719345 R square: 0.8034189706938549


In [15]:
print("\nDeepAR")
training_params = {'max_epochs': 100, 'n_trials': 30}
saving_dir = pathForSavingModels('DeepAR', **dl_searchCV_params)
deep_ar, val_dataloader = DEEPAR(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    saving_dir,
    training_params
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
deepar_predictions = deep_ar.predict(val_dataloader)

model_eval(actuals, deepar_predictions, predictions_x_axis, stdout=True, vis=True)

c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



DeepAR


c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES

Epoch 11: 100%|██████████| 39/39 [00:02<00:00, 13.26it/s, loss=6.99, v_num=0, train_loss_step=6.960, train_loss_epoch=6.970, val_loss=8.880]
MAPE: 0.08164822310209274 R square: 0.47812522763567344


# Transformer

In [16]:
print("\nTFT")
training_params = {'max_epochs': 50, 'n_trials': 10}
saving_dir = pathForSavingModels('TFT', **dl_searchCV_params)
tft, val_dataloader = TFT(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_unknown_categoricals,
    time_varying_known_categoricals,
    time_varying_known_reals,
    time_varying_unknown_reals,
    batch_size,
    saving_dir,
    training_params
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
tft_predictions = tft.predict(val_dataloader)

model_eval(actuals, tft_predictions, predictions_x_axis, stdout=True, vis=True)

c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



TFT


c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\pytorch_lightning\trainer\connectors\callback_connector.py:172: LightningDeprecationWarning:

Setting `Trainer(weights_summary=None)` is deprecated in v1.5 and will be removed in v1.7. Please set `Trainer(enable_model_summary=False)` instead.

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES

Epoch 26: 100%|██████████| 39/39 [00:03<00:00, 10.06it/s, loss=0.0271, v_num=0, train_loss_step=0.0309, train_loss_epoch=0.029, val_loss=0.045]  
MAPE: 0.044981442391872406 R square: 0.7995169166863343
